In [ ]:
!pip install diffusers --upgrade

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors

In [3]:
!pip install --upgrade accelerate

In [ ]:
from diffusers import DiffusionPipeline
import torch

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

In [ ]:
# n_steps: Total number of steps in the image generation process.
# This defines how many iterations the model will go through to refine the generated image.
n_steps = 40

# high_noise_frac: Fraction of the total steps dedicated to high noise processing.
# A higher value means more steps are used in the initial, more random phase of image generation,
# contributing to the overall structure and randomness of the image.
high_noise_frac = 0.8

prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

In [ ]:
# Show the image
from IPython.display import Image, display

# Display the image
display(image)

In [ ]:
# Specify the file path to save the image
file_path = 'generated_image.png'

# Save Image
image.save(file_path)